In [2]:
import os
import cv2
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
from pathlib import Path

VIDEO_PATH = "D:/Szakdolgozat/githubclone/kozlekedesi_tabla_felismero/Videos/0002.mp4"
OUTPUT_CSV = "D:/Szakdolgozat/githubclone/kozlekedesi_tabla_felismero/Detections/0002.csv"
MODEL_PATH = "D:/Szakdolgozat/githubclone/kozlekedesi_tabla_felismero/mentett_model/traffic_sign_model.h5"
IMG_SIZE = (256, 256)
THRESHOLD = 0.7
TIMESTEP = 0.5


model = load_model(MODEL_PATH)

label_path = Path("D:/Szakdolgozat/githubclone/kozlekedesi_tabla_felismero/MultiLabelFrames/labels.csv")
df_labels = pd.read_csv(label_path)
sign_names = df_labels.columns.drop("filename").tolist()

cap = cv2.VideoCapture(VIDEO_PATH)
fps = cap.get(cv2.CAP_PROP_FPS)
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
duration = frame_count / fps

timestamps = np.arange(0, duration, TIMESTEP)
results = []

for t in timestamps:
    cap.set(cv2.CAP_PROP_POS_MSEC, t * 1000)
    ret, frame = cap.read()
    if not ret:
        results.append([t] + [0] * len(sign_names))
        continue

    frame = cv2.resize(frame, IMG_SIZE)
    image = tf.convert_to_tensor(frame / 255.0, dtype=tf.float32)
    image = tf.expand_dims(image, axis=0)

    preds = model.predict(image)[0]
    labels = (preds > THRESHOLD).astype(int).tolist()

    results.append([t] + labels)

cap.release()

df_out = pd.DataFrame(results, columns=["timestamp"] + sign_names)
Path(OUTPUT_CSV).parent.mkdir(parents=True, exist_ok=True)
df_out.to_csv(OUTPUT_CSV, index=False)
print("✅ Kész:", OUTPUT_CSV)


1/1 [==============================] - 0s 20ms/step
✅ Kész: D:/Szakdolgozat/githubclone/kozlekedesi_tabla_felismero/Detections/0002.csv
